In [1]:
import mediapipe as mp
import cv2

mpDraw = mp.solutions.drawing_utils             
mpHands = mp.solutions.hands 

mpDraw_point = mpDraw.DrawingSpec((255, 255, 255), 3 , 3)
mpDraw_line = mpDraw.DrawingSpec((0, 255, 255), 2, 2)

hands = mpHands.Hands(static_image_mode = False, max_num_hands = 2,
                                                 min_detection_confidence = 0.9,
                                                 min_tracking_confidence = 0.5)    


In [6]:
cap = cv2.VideoCapture(0)
while True:
    hasFrame, frame = cap.read()
    if not hasFrame: break
    (imgHeight, imgWidth, _) = frame.shape
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(imgRGB) 
    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS, mpDraw_point, mpDraw_line)
            xPos = int(handLms.landmark[8].x * imgWidth)
            yPos = int(handLms.landmark[8].y * imgHeight)
            cv2.circle(frame, (xPos, yPos), 8, (255, 0, 0), cv2.FILLED) 
            
            '''
            for i, lm in enumerate(handLms.landmark):
                xPos = int(lm.x * imgWidth)
                yPos = int(lm.y * imgHeight)
                cv2.putText(frame, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2)
                if i==8:
                    cv2.circle(frame, (xPos, yPos), 8, (255, 0, 0), cv2.FILLED) 
            '''
    cv2.imshow("Hand Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break
cap.release()
cv2.destroyAllWindows()


In [7]:
import mediapipe as mp
import cv2

mpDraw = mp.solutions.drawing_utils             
mpPoses = mp.solutions.pose

mpDraw_point = mpDraw.DrawingSpec((255, 255, 255), 3 , 3)
mpDraw_line = mpDraw.DrawingSpec((0, 255, 255), 2, 2)

poses = mpPoses.Pose(static_image_mode = False, 
                                            min_detection_confidence=0.5, 
                                            min_tracking_confidence=0.5)


In [10]:
cap = cv2.VideoCapture(0)
while True:
    hasFrame, frame = cap.read()
    if not hasFrame: break
    (imgHeight, imgWidth, _) = frame.shape
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = poses.process(imgRGB) 
    if result.pose_landmarks:
        mpDraw.draw_landmarks(frame, result.pose_landmarks, mpPoses.POSE_CONNECTIONS, mpDraw_point, mpDraw_line)
        for i, lm in enumerate(result.pose_landmarks.landmark):
            xPos = int(lm.x * imgWidth)
            yPos = int(lm.y * imgHeight)
            cv2.putText(frame, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2)
            if i==11 or i==12:
                cv2.circle(frame, (xPos, yPos), 8, (255, 0, 0), cv2.FILLED) 

    cv2.imshow("Hand Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break
cap.release()
cv2.destroyAllWindows()


In [3]:
import mediapipe as mp
import cv2

mpDraw = mp.solutions.drawing_utils             
mpFaces = mp.solutions.face_detection

mpDraw_point = mpDraw.DrawingSpec((255, 255, 255), 3 , 3)
mpDraw_line = mpDraw.DrawingSpec((0, 255, 255), 2, 2)

faces = mpFaces.FaceDetection(min_detection_confidence=0.5, 
                                                            model_selection=0)
cap = cv2.VideoCapture(0)
while True:
    hasFrame, frame = cap.read()
    if not hasFrame: break
    (imgHeight, imgWidth, _) = frame.shape
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = faces.process(imgRGB) 
    if result.detections:
        for detection in result.detections:
            #mpDraw.draw_detection(frame, detection, mpDraw_point, mpDraw_line)
            dldr = detection.location_data
            
            x = int(dldr.relative_bounding_box.xmin * imgWidth)
            y = int(dldr.relative_bounding_box.ymin * imgHeight)
            w = int(dldr.relative_bounding_box.width * imgWidth)
            h = int(dldr.relative_bounding_box.height * imgHeight)
            re = dldr.relative_keypoints[0]
            le = dldr.relative_keypoints[1]
            nt = dldr.relative_keypoints[2]
            mc = dldr.relative_keypoints[3]
            ra = dldr.relative_keypoints[4]
            la = dldr.relative_keypoints[5]
            re_x = int(re.x * imgWidth) ; re_y = int(re.y * imgHeight)
            cv2.circle(frame, (re_x, re_y), 8, (0, 0, 255), cv2.FILLED)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)

    cv2.imshow("Face Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break
cap.release()
cv2.destroyAllWindows()


In [10]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt

mpSegmentation = mp.solutions.selfie_segmentation

Seg = mpSegmentation.SelfieSegmentation(model_selection=1)

bg = np.ones((480,640,3), dtype=np.uint8)*255 

cap = cv2.VideoCapture(0)
while True:
    hasFrame, frame = cap.read()
    if not hasFrame: break
    frame = cv2.resize(frame,(640,480)) 
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = Seg.process(imgRGB) 
    condition = np.stack((result.segmentation_mask,) * 3, axis=-1) > 0.1
    output_image = np.where(condition, frame, bg)
    
    cv2.imshow("Selfie Segmentation", output_image)
    if cv2.waitKey(1) & 0xFF == ord('q'): break
cap.release()
cv2.destroyAllWindows()
